In [1]:
import logging
logging.basicConfig(level=logging.INFO)

In [2]:
from cicada.communicator import SocketCommunicator

def main(communicator):
    print(f"Hello from player {communicator.rank}!")

SocketCommunicator.run(world_size=2, fn=main);

Hello from player 1!Hello from player 0!



In [3]:
logging.getLogger("cicada.communicator").setLevel(logging.INFO)

SocketCommunicator.run(world_size=2, fn=main)

logging.getLogger("cicada.communicator").setLevel(logging.WARNING) # Put things back the way they were

INFO:cicada.communicator.socket.connect:Comm world player 0 listening to tcp://127.0.0.1:65298 for connections.
INFO:cicada.communicator.socket.connect:Comm world player 1 listening to tcp://127.0.0.1:65300 for connections.
INFO:cicada.communicator.socket.connect:Comm world player 1 direct connect with ['tcp://127.0.0.1:65298', 'tcp://127.0.0.1:65300'].
INFO:cicada.communicator.socket.connect:Comm world player 0 direct connect with ['tcp://127.0.0.1:65298', 'tcp://127.0.0.1:65300'].
INFO:cicada.communicator.socket.connect:Comm world player 0 tcp://127.0.0.1:65298 accepted connection from tcp://127.0.0.1:65304
INFO:cicada.communicator.socket.connect:Comm world player 1 tcp://127.0.0.1:65304 connected to player 0 tcp://127.0.0.1:65298
INFO:cicada.communicator.socket:Comm world player 0 communicator ready.
INFO:cicada.communicator.socket:Comm world player 1 communicator ready.


Hello from player 0!Hello from player 1!



INFO:cicada.communicator.socket:Comm world player 1 communicator freed.
INFO:cicada.communicator.socket:Comm world player 0 communicator freed.
INFO:cicada.communicator.socket:Comm world player 0 result: None
INFO:cicada.communicator.socket:Comm world player 1 result: None


In [4]:
from cicada.logging import Logger

def main(communicator):
    log = Logger(logger=logging.getLogger(), communicator=communicator)
    log.info(f"Hello from player {communicator.rank}!")

SocketCommunicator.run(world_size=2, fn=main);

INFO:root:Hello from player 0!
INFO:root:Hello from player 1!


In [5]:
def main(communicator):
    log = Logger(logger=logging.getLogger(), communicator=communicator)
    log.warning(f"Warning message from player {communicator.rank}!")

SocketCommunicator.run(world_size=2, fn=main);

In [6]:
def main(communicator):
    log = Logger(logger=logging.getLogger(), communicator=communicator)
    log.error(f"Error message from player {communicator.rank}!")

SocketCommunicator.run(world_size=2, fn=main);

ERROR:root:Error message from player 0!
ERROR:root:Error message from player 1!


In [7]:
def main(communicator):
    log = Logger(logger=logging.getLogger(), communicator=communicator)
    log.critical(f"Critical message from player {communicator.rank}!")

SocketCommunicator.run(world_size=2, fn=main);

CRITICAL:root:Critical message from player 0!
CRITICAL:root:Critical message from player 1!


In [8]:
logging.getLogger().setLevel(logging.DEBUG) # Don't filter DEBUG messages

def main(communicator):
    log = Logger(logger=logging.getLogger(), communicator=communicator)
    log.debug(f"Debug message from player {communicator.rank}!")

SocketCommunicator.run(world_size=2, fn=main);

DEBUG:root:Debug message from player 0!
DEBUG:root:Debug message from player 1!


In [9]:
def main(communicator):
    log = Logger(logger=logging.getLogger(), communicator=communicator)
    log.info(f"Hello only from player {communicator.rank}!", src=1)

SocketCommunicator.run(world_size=5, fn=main);

INFO:root:Hello only from player 1!


In [10]:
def main(communicator):
    log = Logger(logger=logging.getLogger(), communicator=communicator)
    log.info(f"Hello only from player {communicator.rank}!", src=[0, 2, 4])

SocketCommunicator.run(world_size=5, fn=main);

INFO:root:Hello only from player 0!
INFO:root:Hello only from player 2!
INFO:root:Hello only from player 4!


In [11]:
def main(communicator):
    log = Logger(logger=logging.getLogger(), communicator=communicator, sync=False)
    log.info(f"Hello from player {communicator.rank}!")

SocketCommunicator.run(world_size=2, fn=main);

INFO:root:Hello from player 0!
INFO:root:Hello from player 1!


In [12]:
def main(communicator):
    log = Logger(logger=logging.getLogger(), communicator=communicator)
    
    try:
        log.info(f"Player {communicator.rank} starting ...")
        
        # Wait for a message that never arrives ...
        if communicator.rank == 1:
            payload = communicator.recv(src=0, tag=42)
        
    except Exception as e:
        log.error(f"Player {communicator.rank} exception: {e}")

SocketCommunicator.run(world_size=2, fn=main);

INFO:root:Player 0 starting ...
INFO:root:Player 1 starting ...
ERROR:cicada.communicator.socket:Comm world player 1 failed: Timeout('Tag LOGSYNC from player 0 timed-out after 5s')


In [13]:
def main(communicator):
    log = Logger(logger=logging.getLogger(), communicator=communicator)
    
    try:
        log.info(f"Player {communicator.rank} starting ...")
        
        # Wait for a message that will neve arrives
        if communicator.rank == 0:
            payload = communicator.recv(src=1, tag=42)
        
    except Exception as e:
        log.sync = False # Disable coordinated logging
        log.error(f"Player {communicator.rank} exception: {e}")

SocketCommunicator.run(world_size=2, fn=main);

INFO:root:Player 0 starting ...
INFO:root:Player 1 starting ...
ERROR:root:Player 0 exception: Tag 42 from player 1 timed-out after 5s


In [14]:
transcript = logging.getLogger("cicada.communicator.socket.transcript")
transcript.setLevel(logging.DEBUG)

In [15]:
import numpy

from cicada.additive import AdditiveProtocolSuite

def main(communicator):
    protocol = AdditiveProtocolSuite(communicator)
    a_share = protocol.share(src=0, secret=numpy.array(2), shape=())
    b_share = protocol.share(src=1, secret=numpy.array(3), shape=())
    c_share = protocol.add(a_share, b_share)
    c = protocol.reveal(c_share)
    
SocketCommunicator.run(fn=main, world_size=3);

DEBUG:cicada.communicator.socket.transcript:Comm world player 2 --> player 0 PRSZ 7149674431465828228
DEBUG:cicada.communicator.socket.transcript:Comm world player 0 --> player 1 PRSZ 7642590203926755697
DEBUG:cicada.communicator.socket.transcript:Comm world player 1 --> player 2 PRSZ 1619289739644348098
DEBUG:cicada.communicator.socket.transcript:Comm world player 1 <-- player 0 PRSZ 7642590203926755697
DEBUG:cicada.communicator.socket.transcript:Comm world player 0 <-- player 2 PRSZ 7149674431465828228
DEBUG:cicada.communicator.socket.transcript:Comm world player 2 <-- player 1 PRSZ 1619289739644348098
DEBUG:cicada.communicator.socket.transcript:Comm world player 2 --> player 0 GATHER 5045835506588721360
DEBUG:cicada.communicator.socket.transcript:Comm world player 1 --> player 0 GATHER 16732610558458297240
DEBUG:cicada.communicator.socket.transcript:Comm world player 0 --> player 0 GATHER 15115042082372412194
DEBUG:cicada.communicator.socket.transcript:Comm world player 1 --> player

If you want to customize the transcript, you can use a rich set of extra logging fields in your formatting string including: `arrow` (message direction, relative to the local rank), `comm` (communicator name), `dir` (message direction, relative to the local rank), `dst` (message destination), `other` (the other player sending or receiving with the local rank), `payload` (message payload), `rank` (local player), `src` (message source), `tag` (message type), and `verb` ("send" or "receive", depending on whether the local player is sending or receiving).

Because the custom fields used to format the transcript don't exist for other log messages, you'll get exceptions if you try to mix the output from the transcript logger with logging from other modules.  In general, it's a good idea to setup a custom handler for the transcript logger and turn-off propagation, so transcript output can be managed separately from other logging events:

In [16]:
handler = logging.StreamHandler()
transcript.addHandler(handler)
transcript.propagate = False

Now, you can change the default formatting to produce a more compact version of the above:

In [17]:
handler.setFormatter(logging.Formatter("{rank} {dir} {other} {tag} {payload}", style="{"))

SocketCommunicator.run(fn=main, world_size=3);

1 > 2 PRSZ 9204763902030781927
2 > 0 PRSZ 3401604958548994605
0 > 1 PRSZ 4133522825288642455
0 < 2 PRSZ 3401604958548994605
1 < 0 PRSZ 4133522825288642455
2 < 1 PRSZ 9204763902030781927
1 > 0 GATHER 6315519994852129357
2 > 0 GATHER 1803046947363149285
0 > 0 GATHER 10328177131494600595
1 > 1 GATHER 6315519994852129357
2 > 1 GATHER 1803046947363149285
0 < 0 GATHER 10328177131494600595
1 < 1 GATHER 6315519994852129357
0 < 1 GATHER 6315519994852129357
2 > 2 GATHER 1803046947363149285
1 < 2 GATHER 1803046947363149285
2 < 2 GATHER 1803046947363149285
0 < 2 GATHER 1803046947363149285
0 > 1 GATHER 10328177131494600595
0 > 2 GATHER 10328177131494600595
1 < 0 GATHER 10328177131494600595
2 < 0 GATHER 10328177131494600595
1 > 2 GATHER 6315519994852129357
2 < 1 GATHER 6315519994852129357


Note in the preceeding log outputs that an entry is generated both when a message is sent *and* when it's received, and that the rank of the  player logging the event is always on the left, whether they are the sender or the receiver.  With the arrows indicating which direction the message is travelling, this is easy for a person to understand, but it makes programmatically parsing the transcript needlessly difficult.  As an alternative, you can adjust the format to always put the sender on the left and the recipient on the right, regardless of which player is logging the event:

In [18]:
handler.setFormatter(logging.Formatter("From {src} to {dst} {tag} {payload}", style="{"))

SocketCommunicator.run(fn=main, world_size=3);

From 0 to 1 PRSZ 6219750010594219896
From 1 to 2 PRSZ 684994107870686502
From 2 to 0 PRSZ 1337690799535652329
From 1 to 2 PRSZ 684994107870686502
From 0 to 1 PRSZ 6219750010594219896
From 2 to 0 PRSZ 1337690799535652329
From 2 to 0 GATHER 5648035585860036261
From 1 to 0 GATHER 2915093561283164648
From 0 to 0 GATHER 9883614926566678328
From 2 to 1 GATHER 5648035585860036261
From 1 to 1 GATHER 2915093561283164648
From 2 to 0 GATHER 5648035585860036261
From 2 to 2 GATHER 5648035585860036261
From 2 to 1 GATHER 5648035585860036261
From 1 to 0 GATHER 2915093561283164648
From 2 to 2 GATHER 5648035585860036261
From 1 to 1 GATHER 2915093561283164648
From 0 to 0 GATHER 9883614926566678328
From 0 to 1 GATHER 9883614926566678328
From 0 to 2 GATHER 9883614926566678328
From 0 to 1 GATHER 9883614926566678328
From 0 to 2 GATHER 9883614926566678328
From 1 to 2 GATHER 2915093561283164648
From 1 to 2 GATHER 2915093561283164648


Since this format produces two identical events for each message (once when the message is sent, and once when the message is received), you may wish to filter the output to eliminate the duplication, e.g. by only logging messages when they're sent:

In [19]:
def only_sent(record):
    return record.verb == "send"

handler.addFilter(only_sent)

SocketCommunicator.run(fn=main, world_size=3);

From 0 to 1 PRSZ 6917127574682492270
From 1 to 2 PRSZ 7838182651854679122
From 2 to 0 PRSZ 727661295569132226
From 2 to 0 GATHER 18387670686023916122
From 1 to 0 GATHER 9786045974325470479
From 0 to 0 GATHER 8719771487070044193
From 2 to 1 GATHER 18387670686023916122
From 1 to 1 GATHER 9786045974325470479
From 0 to 1 GATHER 8719771487070044193
From 2 to 2 GATHER 18387670686023916122
From 0 to 2 GATHER 8719771487070044193
From 1 to 2 GATHER 9786045974325470479
